In [1]:
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
# VGG16 is a Convolutional Neural Network (CNN) Architecture used for imageNet
#ImageNet is an organization of image database
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
import numpy as np

In [2]:
#Image height and width
IMG_W = 32
IMG_H = 32

In [8]:
baseModel = VGG16(input_shape = (IMG_W, IMG_H, 3), include_top = False)
baseModel.summary()
for layer in baseModel.layers:
		layer.trainable = False
baseModel.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [3]:
def build_model():
    # The 16 in VGG16 refers to it has 16 layers that have weights.
    # It takes input as input_shape 32x32 with 3 channel for rgb image
    #include_top: whether to include the 3 fully-connected layers at the top of the network.
    #When loading a given model, the “include_top” argument can be set to False, in which case the
    #fully-connected output layers of the model used to make predictions is not loaded, allowing a 
    #new output layer to be added and trained.
	baseModel = VGG16(input_shape = (IMG_W, IMG_H, 3), include_top = False)
	baseModel.summary()
	
    #Setting layer.trainable to False moves all the layer's weights from trainable to non-trainable. 
    #This is called "freezing" the layer: the state of a frozen layer won't be updated during training
	for layer in baseModel.layers:
		layer.trainable = False
	baseModel.summary()
	
    #baseModel.input is the Input layer of the VGG network
	inputs = baseModel.input
    #baseModel. output is the symbolic tensor of the output of the model
	x = baseModel.output
    # flatten layer
	x = Flatten()(x)
    # dense layer
	x = Dense(8, activation = 'sigmoid')(x)
    # output layer
	outputs = Dense(2)(x)
	
	model = Model(inputs, outputs)
	model.summary()

	return model

In [13]:
def prepare_data():

	# Load image data
	(trainX, trainY), (testX, testY) = mnist.load_data()
	print(trainX.shape, trainY.shape, testX.shape, testY.shape)
    
#The np. argwhere() is a Numpy library function used to find the indices of array elements that are nonzero, grouped by element. 
#The numpy argwhere() function takes an array-like parameter and returns the indices of the array elements.
	train_indices = np.argwhere(trainY < 2)
	test_indices = np.argwhere(testY < 2)
#squeeze() function is used when we want to remove single-dimensional entries from the shape of an array. 
#Syntax : numpy.squeeze(arr, axis=None )
	train_indices = np.squeeze(train_indices)
	test_indices = np.squeeze(test_indices)

	trainX = trainX[train_indices]
	trainY = trainY[train_indices]
	testX = testX[test_indices]
	testY = testY[test_indices]

#pad() function is used to pad the Numpy arrays.
#Syntax: np.pad(array, pad_width, mode='constant', **kwargs)  
# If we assign a constant value to the mode parameter, padding will be done with a constant value.
	trainX = np.pad(trainX, ((0,0),(2,2),(2,2)), 'constant')
	testX = np.pad(testX, ((0,0),(2,2),(2,2)), 'constant')

#numpy.stack() function is used to join a sequence of same dimension arrays along a new axis.
#Syntax: numpy.stack(arrays, axis=0, out=None)
# The axis parameter specifies the index of the new axis in the dimensions of the result. axis=-1 it will be the last dimension.
	trainX = np.stack((trainX,)*3, axis=-1)
	testX = np.stack((testX,)*3, axis=-1)

	classN = 2
	trainY = to_categorical(trainY, classN)
	testY = to_categorical(testY, classN)
	
	# To convert pixel values from 0-255 into 0-1.    
	trainX = trainX.astype(np.float32)
	testX = testX.astype(np.float32)
	trainX /= 255
	testX /= 255    

	# Preprocess image data to be fit with VGG16
	testX = preprocess_input(testX)
	trainX = preprocess_input(trainX)
	
	return trainX, trainY, testX, testY

In [14]:
def main():
	trainX, trainY, testX, testY = prepare_data()
	model = build_model()

    # the batch size is a number of samples processed before the model is updated
	model.compile(loss = 'mse', optimizer='rmsprop')
	model.fit(trainX, trainY, epochs = 10, batch_size = 32, validation_split = 0.2)
	model.compile(metrics = 'accuracy')
	model.evaluate(testX, testY)  

In [15]:
if __name__ == "__main__":
	main()

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_poo

Epoch 1/10
317/317 [==============================] - 83s 260ms/step - loss: 0.0917 - val_loss: 0.0278
Epoch 2/10
317/317 [==============================] - 83s 262ms/step - loss: 0.0247 - val_loss: 0.0217
Epoch 3/10
317/317 [==============================] - 85s 269ms/step - loss: 0.0196 - val_loss: 0.0266
Epoch 4/10
317/317 [==============================] - 85s 268ms/step - loss: 0.0169 - val_loss: 0.0131
Epoch 5/10
317/317 [==============================] - 88s 277ms/step - loss: 0.0151 - val_loss: 0.0119
Epoch 6/10
317/317 [==============================] - 87s 273ms/step - loss: 0.0142 - val_loss: 0.0110
Epoch 7/10
317/317 [==============================] - 85s 268ms/step - loss: 0.0134 - val_loss: 0.0106
Epoch 8/10
317/317 [==============================] - 87s 276ms/step - loss: 0.0126 - val_loss: 0.0101
Epoch 9/10
317/317 [==============================] - 85s 268ms/step - loss: 0.0121 - val_loss: 0.0132
Epoch 10/10
67/67 [==============================] - 14s 206ms/step - los